In [ ]:
!pip install PyYAML tqdm ultralytics Pillow

In [1]:
import os
import random
import shutil
from PIL import Image
from tqdm import tqdm
import yaml
from ultralytics import YOLO

In [2]:
def remove_white_background(image):
    image = image.convert("RGBA")
    datas = image.getdata()

    newData = []
    for item in datas:
        if item[0] > 200 and item[1] > 200 and item[2] > 200:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)

    image.putdata(newData)
    return image

def place_component(pcb, component, placed_bboxes):
    pcb_w, pcb_h = pcb.size
    comp_w, comp_h = component.size

    # 부품이 PCB 크기보다 큰 경우 배치하지 않음
    if comp_w >= pcb_w or comp_h >= pcb_h:
        return pcb, None

    while True:
        x = random.randint(0, pcb_w - comp_w)
        y = random.randint(0, pcb_h - comp_h)
        bbox = (x, y, x + comp_w, y + comp_h)

        # 기존에 배치된 부품과 겹치지 않는지 확인
        overlap = False
        for existing_bbox in placed_bboxes.values():
            if existing_bbox and (bbox[0] < existing_bbox[2] and bbox[2] > existing_bbox[0] and
                                  bbox[1] < existing_bbox[3] and bbox[3] > existing_bbox[1]):
                overlap = True
                break

        if not overlap:
            # 부품을 PCB 이미지에 배치
            pcb.paste(component, (x, y), component)
            return pcb, bbox

def random_select_and_compose(pcb_path, components_folders):
    pcb = Image.open(pcb_path)
    placed_bboxes = {}

    for folder in components_folders:
        component_files = [f for f in os.listdir(folder) if f.endswith(('.png', '.jpg'))]
        selected_file = random.choice(component_files)
        component = Image.open(os.path.join(folder, selected_file))

        if component.mode != 'RGBA':
            component = remove_white_background(component)

        component = component.resize((int(component.size[0]), int(component.size[1])))

        pcb, bbox = place_component(pcb, component, placed_bboxes)
        placed_bboxes[selected_file] = bbox

    return pcb, placed_bboxes

# 바운딩 박스 좌표를 YOLO 형식으로 변환
def convert_to_yolo_bbox(img_size, bbox):
    dw = 1. / img_size[0]
    dh = 1. / img_size[1]
    x = (bbox[0] + bbox[2]) / 2.0 - 1
    y = (bbox[1] + bbox[3]) / 2.0 - 1
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def generate_yolo_data(pcb_path, components_folders, output_folder, num_images):
    # 디렉토리 생성
    for folder in ['train', 'val']: # train 폴더와 val 폴더 생성
        os.makedirs(os.path.join(output_folder, folder, 'images'), exist_ok=True) # images 폴더 생성
        os.makedirs(os.path.join(output_folder, folder, 'labels'), exist_ok=True) # labels 폴더 생성

    pcb_files = [f for f in os.listdir(pcb_path) if f.endswith(('.png', '.jpg'))]
    for i in tqdm(range(num_images)):
        # PCB 이미지 무작위 선택
        selected_pcb_file = random.choice(pcb_files)
        selected_pcb_path = os.path.join(pcb_path, selected_pcb_file)

        # 합성 이미지 생성
        composed_image, bboxes = random_select_and_compose(selected_pcb_path, components_folders)

        # 파일 저장
        img_filename = f"{i}.jpg"
        txt_filename = f"{i}.txt"
        yolo_text_content = ""
        for component, bbox in bboxes.items():

            if bbox is None:
                continue
            # 파일 이름에서 부품 종류 추출 및 소문자로 변환
            component_type = component.split('_')[0].lower()
            
            # YOLO 양식으로 변환
            yolo_bbox = convert_to_yolo_bbox(composed_image.size, bbox)
            class_index = object_classes[component_type]
            yolo_text_content += f"{class_index} {yolo_bbox[0]} {yolo_bbox[1]} {yolo_bbox[2]} {yolo_bbox[3]}\n"

        # 훈련/검증 데이터 분할
        if i < num_images * 0.8:  # 80%는 훈련 데이터
            output_img_path = os.path.join(output_folder, 'train', 'images', img_filename)
            output_txt_path = os.path.join(output_folder, 'train', 'labels', txt_filename)
        else:  # 나머지 20%는 검증 데이터
            output_img_path = os.path.join(output_folder, 'val', 'images', img_filename)
            output_txt_path = os.path.join(output_folder, 'val', 'labels', txt_filename)

        composed_image.save(output_img_path)
        with open(output_txt_path, "w") as file:
            file.write(yolo_text_content.strip())

In [4]:
pcb_path = '../data/train_data/PCB/' # PCB 이미지 경로
capacitor_folder = '../data/train_data/capacitor/' # Capacitor 이미지 경로
ic_folder = '../data/train_data/IC/' # IC 이미지 경로
resistor_folder = '../data/train_data/resistor/' # Resistor 이미지 경로

In [5]:
object_classes = {"ic": 0, "capacitor": 1, "resistor": 2}
pcb_files = [f for f in os.listdir(pcb_path) if f.endswith(('.png', '.jpg'))]
selected_pcb_file = random.choice(pcb_files)
selected_pcb_path = os.path.join(pcb_path, selected_pcb_file)

In [6]:
generate_yolo_data(pcb_path, [capacitor_folder, ic_folder, resistor_folder], './generate_data', 1000)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:28<00:00,  6.75it/s]


In [10]:
data = {'train':'/home/ubuntu/storage2/dongbeen/Competition_task2/중급_제출파일/generate_data/train/images/', # 생성한 훈련 데이터 이미지 파일의 경로 (상대 경로 말고 절대 경로로 해야함)
        'val':'/home/ubuntu/storage2/dongbeen/Competition_task2/중급_제출파일/generate_data/val/images/', # 생성한 검증 데이터 이미지 파일의 경로
       'names':['ic', 'capacitor', 'resistor'],
       'nc':3}
with open('./data.yaml', 'w') as f: # yaml 파일이 저장될 경로
    yaml.dump(data, f)

In [11]:
model = YOLO('yolov8n.pt') # yolov8n 모델구조와 가중치를 다운로드 받습니다.
model.train(data="./data.yaml", epochs=100, batch=64, imgsz=480, patience=20, device=0, exist_ok=True,  # GPU가 하나인 경우 device를 0으로 설정, data는 위에서 생성된 yaml 파일의 경로
            optimizer="Adam", lr0=0.001, nms=True, degrees=0.5, flipud=0.5, translate=0.1, scale=0.2, fliplr=0.5, mosaic=0.3)

New https://pypi.org/project/ultralytics/8.0.209 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.81 🚀 Python-3.11.4 torch-2.0.1+cu117 CUDA:3 (NVIDIA RTX A5000, 24256MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./data.yaml, epochs=100, patience=20, batch=64, imgsz=480, save=True, save_period=-1, cache=False, device=3, workers=8, project=None, name=None, exist_ok=True, pretrained=False, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fal

KeyboardInterrupt: 